In [148]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential

import numpy as np
import pandas as pd

In [149]:
#import the data ---> these lines are just testing.
test = pd.read_csv("data_four_labels/coffee/_wijo648v0g3208.csv")
print(test)

test2=pd.read_csv("data_four_labels/coffee/2sGQuduhAf41354.csv")
print(test2)

final_df=pd.concat([test,test2])
print(final_df)

      class        x1        y1        z1        v1        x2        y2  \
0    coffee  0.490560  0.290803 -0.457349  0.999989  0.507861  0.251743   
1    coffee  0.490317  0.296208 -0.467214  0.999987  0.507768  0.253555   
2    coffee  0.490271  0.300322 -0.470750  0.999986  0.507730  0.255209   
3    coffee  0.490268  0.302454 -0.487628  0.999985  0.507728  0.256161   
4    coffee  0.490803  0.303688 -0.505418  0.999985  0.507995  0.256749   
..      ...       ...       ...       ...       ...       ...       ...   
110  coffee  0.496863  0.302861 -0.649061  0.999965  0.513863  0.255302   
111  coffee  0.496631  0.301382 -0.643764  0.999955  0.513532  0.254744   
112  coffee  0.496510  0.300023 -0.687388  0.999919  0.513147  0.254390   
113  coffee  0.496565  0.299678 -0.744448  0.999876  0.513067  0.254359   
114  coffee  0.496665  0.299629 -0.655965  0.999852  0.513007  0.254376   

           z2        v2        x3  ...       z73  v73       x74       y74  \
0   -0.412709  0.99998

In [150]:
import glob

df = pd.DataFrame()
names = ["coffee", "dog", "door", "milk"]
for label in names:
    files = glob.glob(f"data_four_labels/{label}/*.csv")
    for f in files:
        csv = pd.read_csv(f)
        df = pd.concat([df, csv])

print(df)

     class        x1        y1        z1        v1        x2        y2  \
0   coffee  0.515050  0.337479 -0.798418  0.999485  0.524108  0.302156   
1   coffee  0.514914  0.337661 -0.793975  0.999504  0.524079  0.302088   
2   coffee  0.514846  0.337965 -0.765427  0.999515  0.524072  0.302107   
3   coffee  0.514829  0.337725 -0.752842  0.999517  0.524080  0.301461   
4   coffee  0.514829  0.337339 -0.762397  0.999538  0.524121  0.300769   
..     ...       ...       ...       ...       ...       ...       ...   
75    milk  0.497894  0.306149 -0.517604  0.999934  0.515566  0.259491   
76    milk  0.497847  0.305891 -0.508041  0.999933  0.515531  0.259391   
77    milk  0.497788  0.305599 -0.499464  0.999932  0.515466  0.259253   
78    milk  0.497708  0.305279 -0.495048  0.999933  0.515386  0.259108   
79    milk  0.497436  0.305055 -0.492145  0.999933  0.515184  0.258973   

          z2        v2        x3  ...  z73  v73  x74  y74  z74  v74  x75  y75  \
0  -0.776017  0.999213  0.5308

#### separating the features from the labels (just some testing, not actually used)

In [151]:
df_features = df.copy()

df_labels = df_features.pop("class")

df_features = np.array(df_features)

print(df_features.shape)
print(df_labels.shape)
df_labels.head()

(13443, 300)
(13443,)


0    coffee
1    coffee
2    coffee
3    coffee
4    coffee
Name: class, dtype: object

I"ll also remap every string to an int
per example, 
through a dictionary

In [152]:
#should be between 0 and 3
dicti={"coffee": 0, "dog": 1, "door": 2, "milk": 3}
# for element in df_labels:
#     element = dicti[element]

def change_label(x):
    return dicti[x]

df_labels = df_labels.apply(change_label)
df_labels.head()



0    0
1    0
2    0
3    0
4    0
Name: class, dtype: int64

In [153]:
#I have to make each label a single vector 
# I can just rotate (transpose, since now I have a single rolumn, I will make it into many rows)! 
df_labels = np.array([df_labels]).T

print(df_labels.shape)

(13443, 1)


#### train_test_split

##### I'll split into train and val, and then tensorflow will split for me in train and test

In [154]:
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(df_features, df_labels, test_size=0.2)

print(X_train.shape)
print(X_train)
print(y_train.shape)
print(y_train)

(10754, 300)
[[ 0.62516278  0.31280386 -0.57702363 ...  0.37486142 -0.03240779
   0.        ]
 [ 0.5000419   0.43496644 -0.4292005  ...  0.          0.
   0.        ]
 [ 0.48610997  0.27184641 -0.47398481 ...  0.85514456 -0.04888245
   0.        ]
 ...
 [ 0.32949692  0.29826134 -0.80669653 ...  0.          0.
   0.        ]
 [ 0.51242888  0.34741375 -0.45631069 ...  0.67133045 -0.0255616
   0.        ]
 [ 0.32880014  0.29038998 -0.72144747 ...  0.          0.
   0.        ]]
(10754, 1)
[[2]
 [3]
 [0]
 ...
 [1]
 [3]
 [1]]


## creating the model

In [155]:
model = Sequential([
    Dense(units=256, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=4, activation='softmax')
])

In [156]:
#cost function
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])


In [157]:
#fitting
model.fit(X_train,y_train, epochs=10, validation_split=0.1)

Epoch 1/10
303/303 [==============================] - 3s 5ms/step - loss: 0.8560 - accuracy: 0.6355 - val_loss: 0.7181 - val_accuracy: 0.7221
Epoch 2/10
303/303 [==============================] - 1s 4ms/step - loss: 0.6536 - accuracy: 0.7348 - val_loss: 0.6282 - val_accuracy: 0.7779
Epoch 3/10
303/303 [==============================] - 1s 3ms/step - loss: 0.5665 - accuracy: 0.7746 - val_loss: 0.5710 - val_accuracy: 0.7751
Epoch 4/10
303/303 [==============================] - 1s 4ms/step - loss: 0.4916 - accuracy: 0.8057 - val_loss: 0.4769 - val_accuracy: 0.8160
Epoch 5/10
303/303 [==============================] - 1s 4ms/step - loss: 0.4273 - accuracy: 0.8321 - val_loss: 0.4522 - val_accuracy: 0.8216
Epoch 6/10
303/303 [==============================] - 1s 4ms/step - loss: 0.4011 - accuracy: 0.8414 - val_loss: 0.3965 - val_accuracy: 0.8476
Epoch 7/10
303/303 [==============================] - 1s 3ms/step - loss: 0.3607 - accuracy: 0.8583 - val_loss: 0.3321 - val_accuracy: 0.8708
Epoch 

In [160]:
model.evaluate(X_val,y_val)
model.summary()

85/85 [==============================] - 0s 3ms/step - loss: 0.2678 - accuracy: 0.9093
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 256)               77056     
                                                                 
 dense_27 (Dense)            (None, 64)                16448     
                                                                 
 dense_28 (Dense)            (None, 4)                 260       
                                                                 
Total params: 93,764
Trainable params: 93,764
Non-trainable params: 0
_________________________________________________________________


## save the model

In [162]:
model.save("saved_model")

INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets
